In [7]:
# https://github.com/jmtomczak/git_flow/blob/main/models/idf.py
# https://arxiv.org/pdf/2011.15056

import os
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from pytorch_model_summary import summary
import yaml
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

from util import samples_generated, samples_real, plot_curve
import idf
from train import evaluation, training 
from data import load_data
from neural_networks import nnetts

batch_size = 1000
train_data, val_data, test_data = load_data('mnist')
# Create data loaders
training_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

result_dir = 'results/exp_test'
if not(os.path.exists(result_dir)):
    os.mkdir(result_dir)
name = 'idf-4'

D = 784   # input dimension
M = D  # the number of neurons in scale (s) and translation (t) nets
lr = 1e-3 # learning rate
num_epochs = 100 # max. number of epochs
max_patience = 20 # an early stopping is used, if training doesn't improve for longer than 20 epochs, it is stopped
num_flows = 4 # The number of invertible transformations

hyperparameters = {'D': D, 
                   'M': M,
                   'lr': lr,
                   'num_epochs': num_epochs,
                   'max_patience': max_patience,
                   'num_flows': num_flows,
                   'batch_size': batch_size
                    }

with open(result_dir + '/hyperparameters.yaml', 'w') as file:
    yaml.dump(hyperparameters, file)

netts = nnetts(D, M)
model = idf.IDF4(netts, num_flows, D=D)
#print(summary(model, torch.zeros(1, 64), show_input=False, show_hierarchical=False))
optimizer = torch.optim.Adamax([p for p in model.parameters() if p.requires_grad == True], lr=lr)

IDF by JT.


In [8]:
# Training procedure
nll_val = training(name=name, result_dir = result_dir, max_patience=max_patience, num_epochs=num_epochs, model=model, optimizer=optimizer,
                       training_loader=training_loader, val_loader=val_loader)

Epoch: 0, val nll=1835.0413958333334
saved!
Epoch: 1, val nll=1794.1198958333334
saved!
Epoch: 2, val nll=1752.4939375
saved!
Epoch: 3, val nll=1712.9669791666668
saved!
Epoch: 4, val nll=1670.66675
saved!
Epoch: 5, val nll=1629.8003125
saved!
Epoch: 6, val nll=1588.7243541666667
saved!
Epoch: 7, val nll=1547.7170833333332
saved!
Epoch: 8, val nll=1511.2812916666667
saved!
Epoch: 9, val nll=1466.6698958333334
saved!
Epoch: 10, val nll=1428.7598333333333
saved!
Epoch: 11, val nll=1388.7761666666668
saved!
Epoch: 12, val nll=1349.5886666666668
saved!
Epoch: 13, val nll=1320.7963125
saved!
Epoch: 14, val nll=1274.9585625
saved!
Epoch: 15, val nll=1234.1487291666667
saved!
Epoch: 16, val nll=1199.0406875
saved!
Epoch: 17, val nll=1164.0422916666666
saved!
Epoch: 18, val nll=1128.8340833333334
saved!
Epoch: 19, val nll=1091.5494375
saved!
Epoch: 20, val nll=1059.637375
saved!
Epoch: 21, val nll=1027.4694791666666
saved!
Epoch: 22, val nll=982.7489479166667
saved!
Epoch: 23, val nll=954.9231

In [10]:
with open(result_dir + '/train_loss.txt', "w") as file:
    for item in nll_val:
        file.write(f"{item}\n")

test_loss = evaluation(name=result_dir + '/' + name, test_loader=test_loader)
f = open(result_dir + '/test_loss.txt', "w")
f.write(str(test_loss))
f.close()

samples_generated(result_dir + '/' + name, test_loader, 28)
plot_curve(result_dir + '/' + name, nll_val)

FINAL LOSS: nll=515.314859375
